# *Case Study: Electric Scooters Sharing*

**ASSIGNMENT #2 - PROGRAMMING, ALGORITHMS AND DATA STRUCTURES**

## Importing libraries

We will be using three libraries.
- ***math*** let us perform some basic mathematical operations such as the square root.
- ***names*** let us give random names to our users
- ***random*** let us randomize a lot of variables because inputting all of them by hand will take too long (we will have to work with 10.000 subscribers, 5.000 scooters and 100 stations).

In [1]:
pip install names


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Importing the libraries needed
import math
import names
import random

## User class

The renting area is a square with a side of 10 km. Within it, there are squared stations with a side of 50 metres.

All the **location** variables in the code are tuples of two float numbers. Each tuple expresses a precise point in the renting area.

***subscription_type***:
- Standard: 5 DKK/min
- With Insurance: 3 DKK/min + 10 DKK to unlock.

With ***calculate_ride_cost()*** we use the *subscription_type* of a user to calculate the cost of every ride.

Using ***choice()***, we capture the user's decision to either accept or decline the suggested scooter. 

In [3]:
class User():

    all_users = [] # Initializing a list which contains all the users
    user_count = 0 # Keeps track of the number of users 

    def __init__(self):
        self.name = names.get_first_name() # Picks a random name from the library
        self.surname = names.get_last_name() # Picks a random surname from the library
        self.payment_method = random.choice(["Card", "ApplePay", "GooglePay", "SamsungPay", "Klarna"]) # Picks a random payment_method among those options
        self.subscription_type = random.choice(["Standard", "Premium"]) # Picks a random subscription_type
        self.user_location = (random.uniform(0, 10000), random.uniform(0, 10000)) # Our area is a square with a side of 10 km

        User.all_users.append(self) 
        User.user_count += 1
        self.userID = User.user_count # The first user will have userID = 1, second one userID = 2 and so on
        self.km_history = 0.0 # Kilometres covered by each user since he/she created the account
        self.money_spent = 0.0 # DKK spent by each user since he/she created the account
        self.number_of_rents = 0 # Number of rents made by each user since he/she created the account
    
    def calculate_ride_cost(self, ride_time): # Calculates the ride cost depending on the subscription_type of the user
        return (ride_time * 3) + 10 if self.subscription_type == "Premium" else ride_time * 5 # ride_time is in minutes

    def choice(self):
        if random.choice([0, 0, 0, 0, 1, 1, 1, 1, 1, 1]) == 1: # 60 % of probability to accept the scooter
            return True # Accepting (1) the scooter proposed by the program
        return False # Rejecting (0) the scooter proposed by the program


## Scooter class

**status**: 
- 0 if the scooter is NOT available
- 1 if the scooter is available

**residual_autonomy**: when fully charged, each scooter has 50 km of autonomy.

**issues**: 
- 0 if the scooter has NO issues
- 1 if the scooter has issues

***calculate_distance***: is a useful static function which will be used several times in the following code. Given two locations, it returns the Euclidean distance between them.

***sorted_scooters()***: a static function that, given a user location, returns a list of scooters sorted by the nearest to the furthest. For each user, the program will need only the first three elements of the list.

***recharge()***: when a scooter has a residual_autonomy which is below 10 km and it is available, some "runners" hired by our company will instantly go and recharge it.

***mainteinance()***: when a scooter has an issue reported and it is available, some runners will instantly go and fix it.

In [4]:
class Scooter():

    all_scooters = [] # List that contains all the scooters
    scooter_count = 0 # Keeps track of the number of scooters
    recharges_count = 0 # Keeps track of the recharges done by runners
    maintenances_count = 0 # Keeps track of the maintenances done by runners

    def __init__(self):
        self.location = (random.uniform(0, 10000)), (random.uniform(0, 10000)) 
        self.residual_autonomy = 50 # Each scooter initally have full autonomy (50 km)
        self.status = 1 # Each scooter is initially available
        self.issues = random.choice([0, 0, 0, 0, 0, 0, 0, 0, 0, 1]) # 10% of probability that the scooter has any kind of issue

        Scooter.all_scooters.append(self)
        Scooter.scooter_count += 1
        self.scooterID = Scooter.scooter_count # e.g. first scooter generated will have scooterID = 1
        self.name = "Scooter " + str(self.scooterID) # e.g. first scooter generated will be named "Scooter 1"
        self.station = None # The station in which the scooter is
    
    @staticmethod
    def calculate_distance(loc1, loc2): # Useful static function that computes the Euclidean distance between two points in the area
        return math.sqrt((loc1[0] - loc2[0]) ** 2 + (loc1[1] - loc2[1]) ** 2)
    
    @staticmethod
    def sorted_scooters(coord_user): # Sorting algorithm
        available_scooters = [scooter for scooter in Scooter.all_scooters if scooter.status == 1] # List with all the available scooters
        if not available_scooters:
            return None # No available scooters
        sorted_scooters = sorted(available_scooters, key =  lambda scooter: Scooter.calculate_distance(scooter.location, coord_user)) # Sorted list from the nearest scooter to the furthest
        return sorted_scooters[:3]

    def recharge(self): 
        if self.status == 1 and self.residual_autonomy <= 10:
            self.residual_autonomy = 50
            Scooter.recharges_count += 1
        return self.residual_autonomy

    def maintenance(self): 
        if self.status == 1 and self.issues == 1:
            self.issues = 0 
            Scooter.maintenances_count += 1
        return self.issues

## Station class

**location**: represents the station's center. The coordinate randomizer range is intentionally restricted so it doesn’t extend from 0 to 10.000. This is because if the station center were positioned at (20, y) or (x, 9980) then part of the station area would fall outside the designated perimeter—a situation we want to avoid. Rember that each station is a square.

***scooter_within_station()***: function that determines whether a scooter is located within a station. If it is, there's no need to move the scooter to the nearest station.

***take_to_nearest_station()***: function that brings every scooter not within any station to the nearest one. We imagine this to be done by our runners.

In [5]:
class Station():

    all_stations = []
    station_count = 0

    def __init__(self):
        self.location = (random.uniform(int(50 * math.sqrt(2)), 10000 - int(50 * math.sqrt(2))), random.uniform(int(50 * math.sqrt(2)), 10000 - int(50 * math.sqrt(2)))) # Coordinates of the center of the station
        self.scooters = [] # A list which holds the available scooters in each station

        Station.all_stations.append(self)
        Station.station_count += 1 
        self.stationID = Station.station_count # e.g. first station generated will have stationID = 1
        self.name = "Station " + str(self.stationID) # e.g. first station generated will be named "Station 1"
    
    def scooter_within_station(self, scooter):
        if self.location[0] - 25 <= scooter.location[0] <= self.location[0] + 25 and self.location[1] - 25 <= scooter.location[1] <= self.location[1] + 25 and scooter.status == 1:
            scooter.station = self
            self.scooters.append(scooter) # Add the scooter into the list of the station's scooters
        else:
            scooter.station = None # If the scooter is not within any station

    def take_to_nearest_station(self, scooter): # Collaborators will bring scooters outside the stations within the nearest station 
        if scooter.status == 1:
            self.scooters.append(scooter) # The scooter gets appended to the list which contains all the station's scooters
            scooter.location = (random.uniform(self.location[0] - 25, self.location[0] + 25), random.uniform(self.location[1] - 25, self.location[1] + 25)) # The scooter's location gets updated randomly within the range of the nearest station
            scooter.station = self.name # The scooter's belonging station gets updated

## Rental class

**pick_up_location** and **lock_up_location** are set randomly in order to guarantee the program the "anywhere feature".

**distance**: euclidean distance between the pick_up_location and the lock_up_location. Sadly, this is not very realistic but otherwise the code would become much complex.
If the distance intended to be covered by the user is higher than the accepted scooter's residual autonomy, the rental will simply fail.

**time** = calculated by dividing the previously generated distance for the average speed, which we imagined to be a random number chosen from 0.2 * maximum speed of the scooter and 0.8 * maximum speed of the scooter. The maximum speed of the scooter is 70 km/h.

***process_rental()***: updates all the user and scooter info after a rental is completed.


In [6]:
class Rental():

    rentals_done = 0

    def __init__(self, user, scooter):
        self.user = user
        self.scooter = scooter
        self.userID = user.userID # Takes it from the User class
        self.payment_method = user.payment_method # Takes it from the User class        
        self.scooterID = scooter.scooterID # Takes it from the Scooter class
        self.pick_up_location = (user.user_location[0] + random.uniform(-10, 10), user.user_location[1] + random.uniform(-10, 10)) # Anywhere feature
        self.lock_up_location = (round(random.uniform(self.pick_up_location[0] - scooter.residual_autonomy, self.pick_up_location[0] + scooter.residual_autonomy), 2), round(random.uniform(self.pick_up_location[1] - scooter.residual_autonomy, self.pick_up_location[1] + scooter.residual_autonomy), 2)) # Anywhere feature
        self.distance = Scooter.calculate_distance(self.pick_up_location, self.lock_up_location)
        if self.distance > scooter.residual_autonomy:
            raise ValueError(f"intended distance exceeds the {scooter.name} residual autonomy.\n")
        self.time = (self.distance / random.uniform(0.2 * 70, 0.8 * 70)) * 60 # In minutes
        self.ride_cost = user.calculate_ride_cost(self.time) # Takes in consideration the two subscription_types

        Rental.rentals_done += 1
        self.rentalID = Rental.rentals_done # e.g. first rental will have rentalID = 1
    
    def process_rental(self):
        # Updating User infos
        self.user.km_history += self.distance
        self.user.money_spent += self.ride_cost
        self.user.number_of_rents += 1 
        # Updating Scooter infos
        self.scooter.location = self.lock_up_location
        self.scooter.status = 1
        self.scooter.residual_autonomy -= self.distance
        
        print(f"Rental #{self.rentalID} Processed: {self.user.name} {self.user.surname} (ID: {self.user.userID}): used {self.scooter.name} for {self.distance:.2f} km.")
        print(f"Cost: {self.ride_cost:.2f} DKK. Scooter now at {self.lock_up_location} with {self.scooter.residual_autonomy:.2f} km autonomy left.")


## **Simulation**

**online_users**: we imagine that within a day, for example, not the 100% of the users go online but a random percentage that goes from 25% to 75%.

The program is designed to initially propose the *nearest_scooter*; if the user declines, it moves on to suggest the second closest option. If that is declined too, the program offers the third *nearest_scooter*. Finally, as the user declines all the three options, the program will not continue with further suggestions.

A valid reason for a user to decline a proposed scooter can be that it is too distant or that it doesn't have sufficient *residual_autonomy* for covering the user's intended *distance*. If the user accepts the rental of a scooter which does not have the sufficient *residual_autonomy* to take him/her to the desired destination, the *rental* fails.


In [7]:
def simulation(n_users, n_scooters, n_stations):
    # Initialize users, scooters and stations
    online_users = int(random.uniform(0.25, 0.75) * n_users) # Only a certain percentage of users goes online
    users = [User() for i in range(online_users)]
    scooters = [Scooter() for i in range(n_scooters)]
    stations = [Station() for i in range(n_stations)]

    rental_count = 0 # Keeps track of the number of rentals done by users
    declines_count = 0 # Keeps track of the number of scooters declined by users

    print(f"---- Simulation runned: {n_users} users ({online_users} online), {n_scooters} scooters, {n_stations} stations. ----\n")

    # Let the user find the nearest scooter
    for user in users:
        nearest_scooter = Scooter.sorted_scooters(user.user_location)[0] # The nearest scooter is the first to be proposed to the user
        if not nearest_scooter:
            print(f"No available scooters for {user.name} {user.surname} (ID: {user.userID}):.\n")
        elif not user.choice(): # If the user refuses the first nearest scooter
            print(f"{user.name} {user.surname} (ID: {user.userID}) refused to rent first nearest scooter: {nearest_scooter.name}. Distance between them: {0.001 * Scooter.calculate_distance(nearest_scooter.location, user.user_location):.2f} km.")
            declines_count += 1
            nearest_scooter= Scooter.sorted_scooters(user.user_location)[1] # Proceed to propose the second nearest scooter

            if user.choice(): # If the user accepts the second nearest scooter
                try: 
                    rental = Rental(user, nearest_scooter) # Create the rental istance for each user and the scooter he/she chose
                    rental_count += 1 # Update the rental counter
                    nearest_scooter.status = 0 # Becomes unavailable
                    nearest_scooter.station = None
                    rental.process_rental() # Process the rental
                    print(f"Rental duration: {rental.time:.2f} minutes. Payment method: {rental.payment_method}.")

                    nearest_scooter.maintenance() # Perform manteinance for scooters with issues
                    nearest_scooter.recharge() # Perform recharging
                
                    for station in Station.all_stations:
                        station.scooter_within_station(nearest_scooter)
                        if nearest_scooter.station: # Checks if the scooter is already in a station
                            print(f"{nearest_scooter.name} is within station {station.name}.")
                    
                    if not nearest_scooter.station: # If the scooter is not within a station it will be moved by some collaborators at the nearest station
                        nearest_station = min(stations, key=lambda station: Scooter.calculate_distance(station.location, nearest_scooter.location))
                        nearest_station.take_to_nearest_station(nearest_scooter)
                        print(f"{nearest_scooter.name} has been moved to {nearest_station.name}.\n")
                
                except ValueError as e: # Pops up when the user accepts a scooter whose residual autonomy is less than the user's desired distance 
                    print(f"Rental failed for {user.name} {user.surname} (ID: {user.userID}): {e}")
                
                # Won't comment the next section because it's the same as the previous one (except for the update of nearest_scooter)

            else: # If the user refuses also the second nearest scooter
                print(f"{user.name} {user.surname} (ID: {user.userID}) refused to rent the second nearest scooter: {nearest_scooter.name}. Distance between them: {0.001 * Scooter.calculate_distance(nearest_scooter.location, user.user_location):.2f} km.")
                declines_count += 1
                nearest_scooter = Scooter.sorted_scooters(user.user_location)[2] # Proceed to propose the third nearest scooter
                if user.choice(): # If the user accepts the third nearest scooter
                    try:
                        rental = Rental(user, nearest_scooter) 
                        rental_count += 1
                        nearest_scooter.status = 0 
                        nearest_scooter.station = None
                        rental.process_rental()
                        print(f"Rental duration: {rental.time:.2f} minutes. Payment method: {rental.payment_method}.")

                        nearest_scooter.maintenance() 
                        nearest_scooter.recharge() 

                        for station in Station.all_stations:
                            station.scooter_within_station(nearest_scooter)
                            if nearest_scooter.station: 
                                print(f"{nearest_scooter.name} is within station {station.name}.")
                    
                        if not nearest_scooter.station: 
                            nearest_station = min(stations, key=lambda station: Scooter.calculate_distance(station.location, nearest_scooter.location))
                            nearest_station.take_to_nearest_station(nearest_scooter)
                            print(f"{nearest_scooter.name} has been moved to {nearest_station.name}.\n")

                    except ValueError as e:
                        print(f"Rental failed for {user.name} {user.surname} (ID: {user.userID}): {e}")
                                
                # If the user refuses also the third nearest scooter
                else:
                    print(f"{user.name} {user.surname} (ID: {user.userID}) refused to rent the third nearest scooter: {nearest_scooter.name}. Distance between them: {0.001 * Scooter.calculate_distance(nearest_scooter.location, user.user_location):.2f} km.\n") 
                    declines_count += 1 

        else: # If the user accepts the first nearest scooter
            try:                
                rental = Rental(user, nearest_scooter) 
                rental_count += 1
                nearest_scooter.status = 0 
                nearest_scooter.station = None
                rental.process_rental() 
                print(f"Rental duration: {rental.time:.2f} minutes. Payment method: {rental.payment_method}.")

                nearest_scooter.maintenance() 
                nearest_scooter.recharge() 
                
                for station in Station.all_stations:
                    station.scooter_within_station(nearest_scooter)
                    if nearest_scooter.station: 
                        print(f"{nearest_scooter.name} is within station {station.name}.")
                    
                if not nearest_scooter.station:
                    nearest_station = min(stations, key=lambda station: Scooter.calculate_distance(station.location, nearest_scooter.location))
                    nearest_station.take_to_nearest_station(nearest_scooter)
                    print(f"{nearest_scooter.name} has been moved to {nearest_station.name}.\n")
                    
            except ValueError as e:
                print(f"Rental failed for {user.name} {user.surname} (ID: {user.userID}): {e}")
        
    # Some metrics
    total_autonomy = sum(scooter.residual_autonomy for scooter in scooters)
    average_autonomy = total_autonomy / len(scooters)

    print("---- Useful Metrics ----\n")
    print(f"Total rentals: {rental_count}.")
    print(f"Total declines made: {declines_count}.")
    print(f"Average residual autonomy: {average_autonomy:.2f} km.")
    print(f"Recharges made: {Scooter.recharges_count}.")
    print(f"Maintenances performed: {Scooter.maintenances_count}.")

# Finally running the simulation with 10.000 subscribers, 5.000 scooters and 100 stations
simulation(10000, 5000, 100)

---- Simulation runned: 10000 users (4823 online), 5000 scooters, 100 stations. ----

Rental #1 Processed: Dominque Whitcher (ID: 1): used Scooter 538 for 14.82 km.
Cost: 100.15 DKK. Scooter now at (7762.18, 9695.94) with 35.18 km autonomy left.
Rental duration: 30.05 minutes. Payment method: SamsungPay.
Scooter 538 has been moved to Station 66.

Lorrie Ford (ID: 2) refused to rent first nearest scooter: Scooter 3749. Distance between them: 0.08 km.
Rental #2 Processed: Lorrie Ford (ID: 2): used Scooter 2702 for 32.07 km.
Cost: 263.99 DKK. Scooter now at (8614.95, 9674.33) with 17.93 km autonomy left.
Rental duration: 84.66 minutes. Payment method: SamsungPay.
Scooter 2702 has been moved to Station 66.

Rental #3 Processed: Julie Orsini (ID: 3): used Scooter 1058 for 22.23 km.
Cost: 91.85 DKK. Scooter now at (6850.15, 5280.55) with 27.77 km autonomy left.
Rental duration: 27.28 minutes. Payment method: SamsungPay.
Scooter 1058 has been moved to Station 57.

Rental failed for Karla Simp